In [ ]:
class Index(list):
    def __init__(self,val, force_list = False):
        if type(val) == list and not(force_list):
            for v in val:
                self.append(v)
        else:
            self.append(val)

In [ ]:
def make_board(n):
    return [[[[k+1 for k in range(n**2)] for l in range(n)] for j in range(n)] for i in range(n**2)]

def row(idx, n=3):
    a = n*(idx[0]//n)
    b = a+n
    ans = [Index([i,idx[1],j]) for i in range(a,b) for j in range(n)]
    return ans

def column(idx, n=3):
    rem = idx[0]%n
    boxes = [rem+i*n for i in range(n)]
    ans = [Index([i,j,idx[2]]) for i in boxes for j in range(n)]
    return ans

def box(idx, n=3):
    ans = [Index([idx[0],i,j]) for i in range(n) for j in range(n)]
    return ans

def char_to_idx(num, n = 3):
    box_col = num %(n**2)
    box_row = num // (n**2)
    box = n*(box_row//n)+box_col//n
    col = num % n
    row = box_row % n
    return [box,row,col]




In [ ]:

optns = [i+1 for i in range(3**2)]

easy = [[[6,optns,optns],[optns,optns,7],[optns,optns,8]],
        [[optns,optns,optns],[6,5,1],[4,2,optns]],
        [[optns,optns,optns],[8,optns,optns],[6,9,5]],
        [[4,8,3],[optns,optns,optns],[optns,7,optns]],
        [[optns,1,6],[optns,optns,optns],[9,3,8]],
        [[optns,2,optns],[optns,optns,3],[optns,5,optns]],
        [[optns,optns,4],[optns,optns,9],[optns,6,optns]],
        [[3,optns,9],[2,optns,4],[optns,7,optns]],
        [[optns,6,optns],[optns,1,optns],[9,optns,optns]]]
 

hard = [[[6,optns,optns],[optns,3,2],[optns,optns,optns]],
        [[optns,optns,2],[9,optns,optns],[optns,optns,8]],
        [[1,optns,optns],[optns,7,optns],[optns,optns,optns]],
        [[2,optns,optns],[7,optns,3],[optns,8,optns]],
        [[optns,optns,optns],[optns,optns,optns],[optns,9,4]],
        [[4,optns,optns],[9,1,optns],[optns,optns,optns]],
        [[optns,optns,4],[1,2,optns],[optns,optns,optns]],
        [[optns,optns,optns],[7,optns,optns],[optns,optns,optns]],
        [[6,optns,optns],[optns,optns,5],[optns,9,4]]]
		
very_hard = [[[4,optns,optns],[1,optns,optns],[2,optns,optns]],
        [[optns,9,5],[6,optns,optns],[optns,optns,optns]],
        [[optns,optns,optns],[8,5,2],[optns,optns,7]],
        [[optns,9,optns],[optns,8,optns],[optns,optns,optns]],
        [[optns,optns,1],[optns,optns,2],[optns,5,3]],
        [[optns,2,optns],[9,4,optns],[optns,optns,optns]],
        [[9,optns,3],[optns,optns,optns],[optns,optns,6]],
        [[optns,optns,optns],[4,optns,optns],[1,optns,optns]],
        [[optns,optns,optns],[1,7,9],[2,optns,optns]]]
 

super_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'.replace('.','0')

medium = '004300209005009001070060043006002087190007400050083000600000105003508690042910300'

In [ ]:
class Board(list):
    def __init__(self, raw_board, first_pass = False, board_type = 'lists', display = False):
        if board_type == 'lists':
            self.n = len(raw_board[0])
        if board_type == 'string':
            self.n = int(len(raw_board)**(1/4))
            raw_board = self.code_board(raw_board)
        
        tmp_board = make_board(self.n)

        for box in tmp_board:
            self.append(box)
        
        self.missing = self.n**4
        
        self.index_list = []
        
        for b in range(self.n**2):
            for r in range(self.n):
                for c in range(self.n):
                    self.index_list.append(Index([b,r,c]))
                    if type(raw_board[b][r][c]) != list :
                        self.set_value(raw_board[b][r][c],Index([b,r,c]), simplify = first_pass)
        
        if display:
            self.display()

    
    def related(self, idxs):
        if type(idxs)== Index:
            idxs = [idxs]
          
        ans = {}
        
        regions = [box, column, row]
        for region in regions:
            tmp = region(idxs[0])
            if min([tmp == region(idx) for idx in idxs]):
                ans[region.__name__] = tmp
                
        return ans

    def enforce(self, idxs, simplify = True, display = False, enforce_must = True):
        if type(idxs) == Index:
            idxs = [idxs] 
        
        changed = False        
        
        related = self.related(idxs)
        
        tmp_vals = [self.get_value(idx) for idx in idxs]
        vals = [val for val in tmp_vals if type(val)== int]
        
        unsolved = [val for val in tmp_vals if type(val) == list]
        
        if len(unsolved) > 1:
            potential = set()
            for x in unsolved:
                potential = potential.union(set(x))
            if len(potential) == len(unsolved):
                vals = vals + list(potential)

           # if min([unsolved[0]== x for x in unsolved[1:]]) and len(unsolved[0]) == len(unsolved):
           #     vals = vals + unsolved[0]
                
        if len(unsolved) > 0 and enforce_must:
            must = set()
            potential = set()
            for val in unsolved:
                potential = potential.union(set(val))
                
            for region in related:
                can_be = set()
                for idx in [idx for idx in related[region] if idx not in idxs]:
                    tmp = self.get_value(idx)
                    if type(tmp) == list:
                        can_be = can_be.union(set(tmp))
                
                must = must.union(set([val for val in potential if val not in can_be]))
                vals = list(set(vals).union(must))
                
                if len(idxs) == 1 and len(must) == 1:
                    self.set_value(list(must)[0],idxs[0])
                    changed = True
        
        if not(changed):
            for region in related:
                for idx in related[region]:
                    if idx not in idxs:
                        tmp = self.get_value(idx)
                        if type(tmp) == list:
                            for val in vals:
                                try:
                                    tmp.pop(tmp.index(val))
                                    changed = True
                                    if len(tmp) == 1 and simplify:
                                        self.set_value(value = tmp[0],idx = idx)
                                except ValueError:
                                    ''
                                except Exception as e:
                                    print(idx)
                                    print(tmp[0])
                                    raise e
        if display:
            self.display()
        return changed
    
    def get_value(self, idx):
        return self[idx[0]][idx[1]][idx[2]]
        
    
    def set_value(self, value, idx, simplify = True, enforce = True, display = False):
        if type(self.get_value(idx)) == list:
            self.missing = self.missing - 1
        self[idx[0]][idx[1]][idx[2]] = value
        
        if enforce:
            self.enforce(idx, simplify = simplify, enforce_must = simplify)
        if display:
            self.display()
    
    def simplify(self, display = False):
        for b in range(self.n**2):
            for r in range(self.n):
                for c in range(self.n):
                    tmp = self.get_value(Index([b,r,c]))
                    if type(tmp) == list:
                        if len(tmp) == 1:
                            self.set_value(tmp[0],Index([b,r,c]))
        if display:
            self.display()
    
    def make_subsets(self, n, include_solved = False):
        import itertools
        superset = [idx for idx in self.index_list if type(self.get_value(idx)) == list]
        ans =  itertools.combinations(superset, n)
        ans = [combo for combo in ans if len(self.related(combo))>0]
        return ans
        
    
    def solve(self, max_itter = None, brute = False, verbose = ['answer', 'itteration', 'error']):
        """ This function solves the current sudoku puzzle
            
            Options for verbose are:
                answer:        shows the final answer
                itteration:    prints out the chages in the current itteration
                error:         in case of error, displays the board configuration at the time of the error
                value:         prints each value being set
                board:         prints the state of the board at the end of each itteration
                brute:         prints any brute force guesses being made 
        """
        import copy
        min_options_len = self.n**2
        itter = 0
        org_missing = copy.deepcopy(self.missing)
        max_group_size = (self.n**2)-1
        group_size = 1

        while self.missing > 0:
            progress = False
            if group_size > max_group_size:
                print('-'*80+'\n'+"I'm stuck."+'\n'+'-'*80)
                break
            
            # fix this to not do all subsets 
            sub_groups = self.make_subsets(group_size)
            for sub_group in sub_groups:
                if self.enforce(sub_group):
                    if 'value' in verbose:
                        for idx in sub_group:
                            print(f'Value at {idx} set to {self.get_value(idx)}.')
                    progress = True
                    group_size = 1
            
            if not(progress):
                group_size = group_size+1
            
            itter = itter + 1
            if 'itteration' in verbose:
                print('-'*30+'\n'+f"Finished itteration {itter} and have filled out {org_missing - self.missing} of {org_missing} missing numbers.")
                print(f'On next itteration trying subgroups of size {group_size}'+'\n'+'-'*30)
            if 'board' in verbose:
                display_board(tmp)
            if itter == max_itter:
                print(f'Reached maximum number of itterations ({max_itter}).')
                break
                
           
       
        if 'answer' in verbose:
            if self.missing == 0:
                print("Succesfully solved board")
            self.display()
        return self.missing == 0
        
            
    
    def colour_map(self, df):
        import pandas as pd
        c1 = 'background-color: white'
        c2 = 'background-color: lightgrey'
        styles = []

        for i in range(self.n**2):
            tmp = []
            for j in range(self.n):

                if ((((i-(i%self.n))/self.n)+j)%2) ==1:
                    c_tmp = c1
                else:
                    c_tmp = c2
                tmp = tmp + [c_tmp]*self.n

            styles.append(tmp)   

        colour_df = pd.DataFrame(styles,columns = df.columns, index = df.index)
        return colour_df
    
    def code_board(self, raw):
        '''switches from a 1 dimentional representation to a 3 dimentional representation'''
        ans = make_board(self.n)

        for i in range(len(raw)):
            if int(raw[i]) != 0:
                idx = char_to_idx(i,self.n)
                try:
                    ans[idx[0]][idx[1]][idx[2]] = int(raw[i])
                except Exception as e:
                    print([i,idx])
                    raise e
        return ans

    def unwind_board(self):
        '''switches from a 3 dimentional representation to a 2 dimentional representation'''
        ans = [[]]*(self.n**2)
        for box in range(self.n**2):
            for row in range(self.n):
                mult = box - (box % self.n)
                ans[mult + row] = ans[mult+row] +self[box][row]
        return ans

    def display(self):
        import pandas as pd
        from IPython.display import display
        df = pd.DataFrame(self.unwind_board())
        df = df.style.applymap(lambda x: 'font-size: 24pt' if type(x) == int else '')
        df = df.apply(self.colour_map, axis=None)
        df = df.set_properties(**{'width':f'{20*self.n}px','cellLoc':'center'})

        display(df)
        


In [ ]:
b = Board(easy, first_pass = False, display = True)

In [ ]:
%time b.solve()

In [ ]:
hb = Board(hard, board_type = 'lists', display = True)

In [ ]:
%time hb.solve()

In [ ]:
shb = Board(super_hard, board_type = 'string', display = True)

In [ ]:
%time shb.solve(verbose =['answer', 'itteration', 'value'])